In [ ]:
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
from gensim.corpora import Dictionary

In [ ]:
def make_bow(query, articles):
    name = articles.crawling_trg.unique()[0].split(query)[0].strip()

    articles = articles.content.apply(lambda x: x.split()).to_list().copy()
    articles = [[word if not (query in word) else query for word in article] for article in articles]
    articles = [[word if not (name in word) else name for word in article] for article in articles]

    dct = Dictionary(articles)  # fit dictionary
    bow_articles = [dct.doc2bow(article) for article in articles]  # convert corpus to BoW format
    bow_articles = [{k:v for k, v in bow_article} for bow_article in bow_articles]
    
    return name, dct, bow_articles

In [ ]:
def id_by_tf_retrieve(query, name, dct, bow_articles, query_th=0, name_th=0):
    query_id = dct.token2id[query]
    query_tfs = []
    for bow_article in bow_articles:
        query_tfs.append(bow_article[query_id])
    query_tfs = np.array(query_tfs)

    name_id = dct.token2id[name]
    name_tfs = []
    for bow_article in bow_articles:
        name_tfs.append(bow_article[name_id])
    name_tfs = np.array(name_tfs)
    
    tf_pairs = [(q, n) for q, n in zip (query_tfs, name_tfs)]
    ids = {}
    for id, tf_pair in enumerate(tf_pairs):
        query_tf, name_tf = tf_pair[0], tf_pair[1]
        if query_tf >= query_th and name_tf >= name_th:
            ids[id] = query_tfs[id] + name_tfs[id]
    if bool(ids) == False:
        return None, None
    
    doc_id = max(ids, key=ids.get)
    tf_score = max(ids.values())

    return doc_id, tf_score

In [ ]:
    name, dct, bow_articles = make_bow(query, business_news_contents)
    doc_id, tf_score = id_by_tf_retrieve(query, name, dct, bow_articles, query_th=1, name_th=3)

In [ ]:
### 모델과 토크나이저를 SentenceTransformer 클래스로 불러옵니다.
pretrained_model_path = "sbert/training_klue_sts_klue-roberta-base-2022-08-17_23-27-13"
model = SentenceTransformer(pretrained_model_path)

### 유사도를 계산하여 인덱스와 스코어를 반환하는 함수입니다.
def get_indices_and_scores(query, news_contents, top_k):
    with torch.no_grad():
        query_embedding = model.encode(query)
        query_embedding = np.expand_dims(query_embedding, axis=0)
        news_embeddings = model.encode(news_contents.content)
        cos_scores = util.pytorch_cos_sim(query_embedding, news_embeddings).squeeze()

        top_k = min(top_k, len(news_contents.content))
        top_k_results = torch.topk(cos_scores, k=top_k)

        scores = top_k_results.values.squeeze()
        indices = top_k_results.indices.squeeze()

    return indices, scores

In [ ]:
### 유사도 점수에 기반하여 문서를 찾아옵니다.

def retrieve_docs(query, business_name, crawling_result):
    ### crawling_trg에 business_name 이름이 있는 것과 없는 것을 구분하여 데이터를 나눕니다.
    crawling_result['checker'] = crawling_result.crawling_trg.str.find(business_name)

    bussiness_news = crawling_result[crawling_result.checker > -1].copy().reset_index(drop=True)
    bussiness_news['idx_original'] = range(len(bussiness_news))

    org_news = crawling_result[crawling_result.checker == -1].copy().reset_index(drop=True)
    org_news['idx_original'] = range(len(org_news))

    business_news_contents = bussiness_news[['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link']]
    org_news_contents = org_news[['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link']]
    
    
    ### 비지니스명과 쿼리의 term frequency threshold를 충족한 기사 중 term frequency의 합계가 가장 높은 기사를 가지고 옵니다.(중복시 첫 번째)
    name, dct, bow_articles = make_bow(query, business_news_contents)
    doc_id, tf_score = id_by_tf_retrieve(query, name, dct, bow_articles, query_th=1, name_th=3)
    if doc_id == None:
        print(f'{name}: 주목할 만한 기사 없음.')
        print('프로그램을 종료합니다.')
        return None
    else:
        top_of_business_news_contents = business_news_contents.iloc[doc_id].to_list() + list([tf_score])
        top_of_business_news_contents = pd.DataFrame([top_of_business_news_contents], columns=['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link', 'score'])

    ### 기관별로 데이터를 나눕니다.
    org_news_contents_splits = []
    for org in org_news_contents.crawling_trg.unique():
        org_news_contents_split = org_news_contents[org_news_contents.crawling_trg == org].reset_index(drop=True).copy()
        org_news_contents_splits.append(org_news_contents_split)

    ### 각 기관별 수행. 기관명과 쿼리의 term frequency threshold를 충족한 기사 중 term frequency의 합계가 가장 높은 기사를 가지고 옵니다.(중복시 첫 번째)
    tops_of_org_news_contents_splits = []
    for org_news_contents_split in org_news_contents_splits:
        name, dct, bow_articles = make_bow(query, org_news_contents_split)
        doc_id, tf_score = id_by_tf_retrieve(query, name, dct, bow_articles, query_th=2, name_th=3)
        if doc_id == None:
            print(f'{name}: 주목할 만한 기사 없음.')
            print()
            continue
        top_of_org_news_contents_split = org_news_contents_split.iloc[doc_id].to_list() + list([tf_score])
        tops_of_org_news_contents_splits.append(top_of_org_news_contents_split)

    if len(tops_of_org_news_contents_splits) == 0:
        print('검색한 상위 기관에 대하여 주목할 만한 기사 없음.')
        print('프로그램을 종료합니다.')
        return None
    
    tops_of_org_news_contents_splits = pd.DataFrame(tops_of_org_news_contents_splits, columns=['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link', 'score'])
    tops_of_org_news_contents_splits = tops_of_org_news_contents_splits.sort_values('score', ascending=False).reset_index(drop=True).copy()

    ### 가장 높은 점수의 business_name 문서를 쿼리로 하여 
    ### 가장 높은 점수의 기관별 문서 뭉치와 유사도 점수를 계산하고 유사도 점수 상위 5개 문서를 찾아옵니다.
    indices, scores = get_indices_and_scores(top_of_business_news_contents.content.iloc[0], tops_of_org_news_contents_splits, 5)
    result = tops_of_org_news_contents_splits.iloc[list(indices)].copy()
    result['score'] = scores

    return top_of_business_news_contents, tops_of_org_news_contents_splits, result

In [ ]:
crawling_result = pd.read_csv('인공지능_문화체육관광부_클라썸_crawled.csv')
query = '인공지능'
business_name = '클라썸'

In [ ]:
print('문서간 유사도 검사를 수행합니다.')
top_of_business_news_contents, tops_of_org_news_contents_splits, result = retrieve_docs(query, business_name, crawling_result)

# top_of_business_news_contents.to_csv('top_scored_business_news_for_query.csv', index=False, encoding='utf-8-sig')
# tops_of_org_news_contents_splits.to_csv('list_of_top_scored_org_news_for_query_by_org.csv', index=False, encoding='utf-8-sig')
# result.to_csv('top_5_orgs_and_their_news_for_top_scored_business_news.csv', index=False, encoding='utf-8-sig')

# print('문서간 유사도 검사가 완료되었습니다. 다음 파일을 생성했습니다.')
# print('top_scored_business_news_for_query.csv')
# print('list_of_top_scored_org_news_for_query_by_org.csv')
# print('top_5_orgs_and_their_news_for_top_scored_business_news.csv')

In [ ]:
top_of_business_news_contents.to_csv('top_scored_business_news_for_query.csv', index=False, encoding='utf-8-sig')
tops_of_org_news_contents_splits.to_csv('list_of_top_scored_org_news_for_query_by_org.csv', index=False, encoding='utf-8-sig')
result.to_csv('top_5_orgs_and_their_news_for_top_scored_business_news.csv', index=False, encoding='utf-8-sig')

In [ ]:
# top_of_business_news_contents
# tops_of_org_news_contents_splits
for idx, row in result.iterrows():
    print(row)
    print()
    print(row.content)
    print()

# Debugging

In [ ]:
# Traceback (most recent call last):
#   File "main.py", line 87, in <module>
#     main()
#   File "main.py", line 75, in main
#     top_of_business_news_contents, tops_of_org_news_contents_splits, result = retrieve_docs(query, business_name, crawling_result)
#   File "/home/ubuntu/jongmin/Jeonghyeon/codes/news_crawler/module/retrieve.py", line 104, in retrieve_docs
#     doc_id, tf_score = id_by_tf_retrieve(query, name, dct, bow_articles, query_th=2, name_th=3)
#   File "/home/ubuntu/jongmin/Jeonghyeon/codes/news_crawler/module/retrieve.py", line 34, in id_by_tf_retrieve
#     name_tfs.append(bow_article[name_id])
# KeyError: 39
# (jeonghyeon) ubuntu@gpu-1:~/jongmin/Jeonghyeon/codes/news_crawler$ 

In [1]:
import json
import argparse
import pandas as pd
from tqdm import tqdm
from module.crawler import get_news_list, crawl_news
from module.preprocess import preprocess
from module.utils import filter_time
from module.retrieve import retrieve_docs, make_bow, id_by_tf_retrieve

In [2]:
query = '영화'
business_name = '명필름'

In [3]:
def temp(x):
    x = x.split()
    name = ' '.join(x[:-1])
    query = '[SEP]' + x[-1]
    return name+query

crawling_result = pd.read_csv('영화_문화체육관광부_명필름_crawled.csv')
crawling_result['crawling_trg'] = crawling_result.crawling_trg.apply(temp)

dfs = []
for trg in list(crawling_result.crawling_trg.unique()):
    dfs.append(crawling_result[crawling_result.crawling_trg == trg])
    
new_dfs = []
for df in dfs:
    sub_org = df.crawling_trg.unique()[0].split('[SEP]')[0]
    print(sub_org)
    search_sub_org = df.content.str.contains(sub_org, case=False, regex=True)
    search_query = df.content.str.contains(query, case=False, regex=True)
    df = df[search_sub_org & search_query].copy().reset_index(drop=True)
    if len(df) > 0:
        new_dfs.append(df)

국립국어원
국립민속박물관
국립아시아문화전당
국립장애인도서관
국립중앙도서관
국립중앙박물관
국립한글박물관
대한민국역사박물관
한국예술종합학교
해외문화홍보원
게임물관리위원회
국립박물관문화재단
대한체육회
사행산업통합감독위원회
세종학당재단
영상물등급위원회
영화진흥위원회
예술의전당
재단법인국악방송
태권도진흥재단
한국관광공사
한국문학번역원
한국문화예술교육진흥원
한국문화예술위원회
한국문화정보원
한국언론진흥재단
한국영상자료원
한국콘텐츠진흥원
아리랑국제방송
영화진흥위원회 기획개발지원센터
영화진흥위원회 영화관입장권 통합전산망
영화진흥위원회 한국영화아카데미
영화진흥위원회 한국영화해외진출플랫폼
한국저작권위원회
문화재청
국립고궁박물관
국립무형유산원
국립해양문화재연구소
궁능유적본부
한국전통문화대학교
국외소재문화재재단
한국문화재재단
명필름


In [4]:
temp('무슨 일인가 일인가 일').split('[SEP]')[0]

'무슨 일인가 일인가'

In [5]:
crawling_result = pd.concat(new_dfs).reset_index(drop=True)

In [6]:
test = crawling_result[crawling_result.crawling_trg == '영화진흥위원회[SEP]영화']

In [18]:
name_tf = test.content.str.count('영화진흥위원회').tolist()
query_tf = test.content.str.count('영화').tolist()

In [19]:
query_th = 1
name_th = 3

In [20]:
print(name_tf)

[1, 1, 1, 2, 1, 1, 2, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 3, 2, 1, 1, 1, 1, 2, 2, 3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 

In [5]:
for df in new_dfs:
    name, dct, bow_articles = make_bow(query, df)
    doc_id, tf_score = id_by_tf_retrieve(query, name, dct, bow_articles, query_th=2, name_th=3)

국립국어원
국립국어원
국립민속박물관
국립민속박물관
국립아시아문화전당
국립아시아문화전당
39
국립장애인도서관
국립장애인도서관
국립중앙도서관
국립중앙도서관
국립중앙박물관
국립중앙박물관
국립한글박물관
국립한글박물관
대한민국역사박물관
대한민국역사박물관
한국예술종합학교
한국예술종합학교
해외문화홍보원
해외문화홍보원
게임물관리위원회
게임물관리위원회
국립박물관문화재단
국립박물관문화재단
대한체육회
대한체육회
사행산업통합감독위원회
사행산업통합감독위원회
세종학당재단
세종학당재단
영상물등급위원회
영상물등급위원회
영화진흥위원회
영화진흥위원회


KeyError: '영화진흥위원회'

In [ ]:
top_of_business_news_contents, tops_of_org_news_contents_splits, result = retrieve_docs(query, business_name, crawling_result)

In [ ]:
name ='무엇 인가'
''.join(name.split())

In [ ]:
data = pd.read_csv('영화_문화체육관광부_명필름_crawled.csv')

In [ ]:
data.columns
data = data[['crawling_trg', 'content']]

In [ ]:
data = data[data.crawling_trg == f'{business_name} 영화']
len(data)

In [ ]:
search_sub_org = data.content.str.contains(business_name, case=False, regex=True)
search_query = data.content.str.contains(query, case=False, regex=True)
data = data[search_sub_org & search_query].copy().reset_index(drop=True)

In [ ]:
len(data)